# TP9 : apprentissage supervisé

Dans ce TP nous mettons en œuvre une partie de l'algorithme d'apprentissage supervisé des $k$-plus proches voisins vu en cours, dans le but de classer des images contenant des chiffres.

La base de chiffres sur laquelle nous travaillerons (**pas besoin de télécharger quoi que ce soit, c'est déjà chargé, mais je cite mes sources**) est la base [MNIST](https://deepai.org/dataset/mnist). Cette base fournit 4 fichiers (**pas besoin de les télécharger**):

* `train-images-idx3-ubyte.gz`: images d'entraînement (60000 images)
* `train-labels-idx1-ubyte.gz`: classes des images d'entraînement
* `t10k-images-idx3-ubyte.gz`: images de test (10000 images)
* `t10k-labels-idx1-ubyte.gz`: classes des images de test

Je vous fournis les fonctions qui permettent d'obtenir les listes d'images et les listes de classes à partir de ces fichiers, une image étant représentée sous forme de liste de listes de niveaux de gris et une classe sous forme d'un chiffre. Pour celà, j'ai largement emprunté et légèrement modifié le code fourni par le site [https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook](https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook).

Ce qui suit est une classe python (notion hors programme ITC) permettant de charger les données et d'obtenir les listes décrites ci-dessus pour les images et les classes des images. (Vous n'avez pas besoin de comprendre la code, mais il sera utilisé par la suite.)

In [ ]:
import numpy as np # linear algebra
import struct
from array import array

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array('B', file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img.tolist()
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

Le code qui suit utilise la classe précédente pour récupérer les images (quelques secondes d'exécution, soyez patients).

In [ ]:
training_images_filepath = 'train-images-idx3-ubyte'
training_labels_filepath = 'train-labels-idx1-ubyte'
test_images_filepath = 't10k-images-idx3-ubyte'
test_labels_filepath = 't10k-labels-idx1-ubyte'

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath,
                                   test_images_filepath, test_labels_filepath)
(img_entrainement, cl_entrainement), (img_test, cl_test) = mnist_dataloader.load_data()

### Exercice 1 : Afficher quelques images

Pour vérifier que vous avez bien compris ce que contient chacune des listes fabriquées ci-dessus.

Voici le code d'une fonction permettant d'afficher une image.

In [ ]:
from PIL import Image
from IPython.display import display

def affiche_image(img):
    """img : image sous forme d'une liste de listes de niveaux de gris
    affichage de l'image"""
    im = Image.new(size=(len(img[0]), len(img)), mode='L')
    for i in range(len(img)):
        for j in range(len(img[0])):
            im.putpixel((j, i), img[i][j])
    display(im)

Afficher les images d'entraînement 3, 735, 3827 et 52615 et les images tests 54 et 9876.

Afficher maintenant les classes de toutes ces images.

### Exercice 2 : insérer un élément dans une liste ordonnée

Dans le but de programmer l'algorithme des $k$-plus proches voisins, on a besoin de maintenir à jour une liste des voisins les plus proches potentiels. On a donc besoin de pouvoir insérer un élément dans une liste ordonnée dans l'ordre croissant de sorte que la liste reste ordonnée et de même taille (pour celà on supprimera le dernier élément de la liste).

Écrire une fonction `inserer` qui prend en argument un couple `(n, d)` où `d` est un entier et une liste `L` d'éléments de la même forme, supposée ordonnée dans l'ordre croissant selon la valeur de la deuxième composante de ses éléments et telle qu'on suppose que la deuxième composante de son dernier élément est supérieure à `d`, et modifie la liste `L` de telle sorte que `L` à la fin contiennent tous les éléments initialement dans `L[:-1]` ainsi que l'élément `(n, d)`, et est ordonnée  dans l'ordre croissant selon la valeur de la deuxième composante de ses éléments.

(En première approche votre fonction peut être linéaire en la longueur de la liste `L`, nous l'améliorerons au prochain TP.)

In [ ]:
import math

L = [(1,math.inf)]
inserer((4,5), L)
assert(L == [(4, 5)])

L = [('a', 1), ('b', 3), ('c', 5)]
inserer(('d', 4), L)
assert(L == [('a', 1), ('b', 3), ('d', 4)])

L = [('a', 1), ('b', 3), ('c', 5)]
inserer(('d', 2), L)
assert(L == [('a', 1), ('d', 2), ('b', 3)])

L = [('a', 1), ('b', 3), ('c', 5)]
inserer(('d', 0), L)
assert(L == [('d', 0), ('a', 1), ('b', 3)])

### Exercice 3 : Trouver les $k$ plus proches voisins

Je donne ici une variante de la fonction du TP8 permettant d'obtenir la distance euclidienne entre deux images (variante car les images ne doivent pas être fournies par le nom d'un fichier les contenant, mais par une liste de listes de niveaux de gris).

In [ ]:
import math

def distance_euclidienne_image(im1, im2):
    assert(len(im1) == len(im2) and len(im1[0]) == len(im2[0]))
    return math.sqrt(sum([(im1[i][j] - im2[i][j])**2 for i in range(len(im1)) for j in range(len(im1[0]))]))

Écrire une fonction `k_plus_proches_voisins` qui prend en argument :

* une image `im` (sous forme de liste de listes de niveaux de gris),
* une liste `L` d'images (sous la même forme) supposée de longueur supérieure à `k`,
* un entier `k`,
* une fonction qui permet de calculer une distance entre deux images (comme `distance_euclidienne_image` le fait),
* une fonction qui permet d'insérer un couple dans une liste de couples (comme `inserer` le fait),

et qui renvoie une liste de `k` couples `(n, d)` où :

* `n` représente un indice dans la liste `L`,
* `d` est la distance entre les images `im` et `L[n]` calculée par `dist`,
* la liste renvoyée contient les `k` plus proches voisins de `im` dans la liste `L` pour la distance `dist`, dans l'ordre croissant des distances de `im` à ces voisins.

In [ ]:
L = k_plus_proches_voisins(img_test[0], img_entrainement[:100], 3, distance_euclidienne_image, inserer)
assert([n for (n, _) in L] == [84, 96, 52])

L = k_plus_proches_voisins(img_test[54], img_entrainement[1000:1100], 5, distance_euclidienne_image, inserer)
assert([n for (n, _) in L] == [28, 74, 4, 31, 80])

### Exercice 4 : Deviner la classe d'une image

Écrire une fonction `classe_devinee` qui prend en arguments une liste `voisins` sous la même forme que ce que renvoie la fonction `k_plus_proches_voisins` et une liste d'entiers représentant des classes (comme `cl_entrainement`) dans laquelle on peut accéder aux classes des éléments de `voisins`, et renvoie la classe majoritaire de ces éléments.

À vous d'écrire les tests pour vérifier si les images 54 et 9876 sont bien classées si on prend `k=3` et la distance euclidienne entre images.